In [1]:
import os

In [2]:
%pwd

'd:\\Data Science\\NLP\\SentimentAnalysis\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd

'd:\\Data Science\\NLP\\SentimentAnalysis'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvalutationConfig: 
    root_dir: Path
    X_test_data_path: Path
    y_test_data_path: Path
    model_path: Path
    

In [6]:
from SentimentAnalysis.constants import *
from SentimentAnalysis.utils.common import read_yaml
from SentimentAnalysis.utils.common import create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            #self.params = read_yaml(params_filepath)

            create_directories([self.config.dataStore_root])

    #2. To be updated configuration inside config into Configuration

    def get_model_evaluation_config(self) -> ModelEvalutationConfig:
          config = self.config.model_evaluation

          create_directories([config.root_dir])

          model_evaluation_config = ModelEvalutationConfig(
                root_dir=config.root_dir,
                X_test_data_path=config.X_test_data_path,
                y_test_data_path=config.y_test_data_path,
                model_path=config.model_path,
                #metric_file_name=config.metric_file_name

          )

          return model_evaluation_config
    

In [8]:
import os
import pandas as pd
import numpy as np
from SentimentAnalysis.logging import logger
from pathlib import Path
import tensorflow as tf
import joblib

In [11]:
class Modevalutation:
    def __init__(self, config: ModelEvalutationConfig):
        self.config = config
        self.threshold = 0.5
    
    def evaluate_and_compare_model(self):
        X_test = np.load(self.config.X_test_data_path)
        y_test = pd.read_csv(self.config.y_test_data_path)

        model = joblib.load(self.config.model_path)
        _, test_accuracy = model.evaluate(X_test, y_test)
        print("Test set accuracy:", test_accuracy)
    
        # Use model.predict to get the probability predictions
        y_test_pred_prob = model.predict(X_test)

        # Convert probability predictions to binary predictions based on the threshold
        y_test_pred_binary = (y_test_pred_prob > self.threshold).astype(int)

        # Flatten the arrays
        y_test_flat = np.ravel(y_test)
        binary_predictions_flat = np.ravel(y_test_pred_binary)

        # Create the actvsPred DataFrame
        actvsPred = pd.DataFrame({'Actual': y_test_flat, 'Predictions': binary_predictions_flat})

        # Print the actvsPred DataFrame
        actvsPred['diff'] = actvsPred['Actual'] - actvsPred['Predictions']
        print(actvsPred.head(20))
        return actvsPred


In [12]:


try:
    config_manager = ConfigurationManager()
    model_evaluation_config = config_manager.get_model_evaluation_config()
    model_evaluation_config = Modevalutation(config=model_evaluation_config)
    model_evaluation_config.evaluate_and_compare_model()
    
except Exception as e:
    raise e

[2023-08-12 00:00:26,837: INFO: common: YAML file loaded successfully: config\config.yaml]
[2023-08-12 00:00:26,837: INFO: common: Created directory at: dataStore]
[2023-08-12 00:00:26,837: INFO: common: Created directory at: dataStore/model_evaluation]
188/188 [==============================] - 5s 25ms/step - loss: 0.4982 - accuracy: 0.8753
Test set accuracy: 0.875333309173584
188/188 [==============================] - 5s 25ms/step
    Actual  Predictions  diff
0        1            0     1
1        0            0     0
2        0            0     0
3        1            1     0
4        1            1     0
5        1            1     0
6        1            1     0
7        1            1     0
8        1            1     0
9        0            0     0
10       0            0     0
11       0            0     0
12       0            0     0
13       0            0     0
14       0            0     0
15       1            1     0
16       0            0     0
17       1            1